In [ ]:
import glob as gb
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Dense, Flatten,experimental,Conv2D,MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam

In [ ]:
df=keras.preprocessing.image_dataset_from_directory(
    directory='PlantVillage/',
    shuffle=True,
    image_size=(256,256),
    batch_size=32
)

In [ ]:
df

In [ ]:
class_names=df.class_names
class_names

In [ ]:
len(df)

# size

In [ ]:
inputsize=256
batch=32
channel=3

In [ ]:
for image_batch,label_batch in df.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())

In [ ]:
# plt.figure(figsize=(10,10))
# for image_batch,label_batch in df.take(1):
#     for i in range(12):
#         ax=plt.subplot(3,4,i+1)
#         plt.imshow(image_batch[i].numpy().astype('uint8'))
#         plt.title(class_names[label_batch[i]])
#         plt.axis('off')

In [ ]:
train_split=int(len(df)*.80)

In [ ]:
trainds=df.take(train_split)

In [ ]:
trainds

In [ ]:
testds=df.skip(train_split)

In [ ]:
valdf=testds.take(6)
testds=testds.skip(6)

In [ ]:
trainds=trainds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
valdf=valdf.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
testds=testds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_rescale=Sequential([
    layers.experimental.preprocessing.Resizing(256,256),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
data_augmentation=Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2),
            ])

In [ ]:
inp=(batch,inputsize,inputsize,channel)

In [ ]:
model=Sequential([
    resize_rescale,
    data_augmentation,
    Conv2D(32,(3,3),activation='relu',input_shape=inp),
    MaxPooling2D((2,2)),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
  
    Flatten(),
    Dense(64,activation='relu'),
    Dense(3,activation='softmax')
])

In [ ]:
model.build(inp)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
EPOCHS=30

In [ ]:
history=model.fit(
    trainds,
    epochs=EPOCHS,
    batch_size=batch,
    validation_data=valdf
)

In [ ]:
import matplotlib.pyplot as plt2

In [ ]:
plt2.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
max(history.history['val_accuracy'])

In [ ]:
model.evaluate(testds)

In [ ]:
model_version=1
model.save('{model_version}')

In [ ]:
import fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
